In [1]:
import pandas as pd
import numpy as np

In [2]:
import json
import datetime
import random

In [3]:
import config
import request_history

In [4]:
# hurricane sandy: formed: 1 July 2014, dissipated: 9 July 2014
date_ = '2012-09-15T00:00:00.000Z'

In [5]:
# hurricane arthur: formed: 1 July 2014, dissipated: 9 July 2014
# date_ = '2014-06-01T00:00:00.000Z'

In [6]:
# hurricane florence: formed: 31 Aug 2018, dissipated: 18 Sept 2018
# date_ = '2018-08-01T00:00:00.000Z'

In [7]:
# hurricane barry: formed: 11 July 2019, dissipated: 19 July 2019
# date_ = '2019-06-15T00:00:00.000Z'

In [8]:
# hurricane Isaias: formed: 30 July 2020, dissipated: 5 Aug 2020
# date_ = '2020-07-01T00:00:00.000Z'

In [9]:
# hurricane nicholas: formed: 12 Sept 2021, dissipated: 20 Sept 2021
# date_ = '2021-08-15T00:00:00.000Z'

In [24]:
# hurricane terms adopted from the Stowe paper: 
# change the name of the storm per date range
query_terms = "Category 3 OR Category 4 OR landfall OR eyewall OR cleanup OR debris OR garbage OR hurricane OR hurricaneian OR perfectstorm OR sandy OR stormporn OR superstorm OR Tropical Storm"

In [11]:
# update dates iterates by 5 days + random hour selection - 500 tweets filled up in minutes
# so a random hour selection was used to interject variability in the time selection 

def generate_dates(old_end):
    """
    updates the start and end dates to pull tweets from the Twitter API (Academic Permissions)
    arguments:
        - old_end: string formated date to start the window for pulling set of 500 tweets
        
    return:
        - new_start: the previous end date
        - new_end: 5 days from the old end, at a random hour
    """
    hour_increment = random.randint(0, 23)
    new_start = old_end

    old_date = datetime.datetime.strptime(old_end, '%Y-%m-%dT%H:%M:%S.%fZ')
    new_end_dtg = old_date + datetime.timedelta(days=3, hours=hour_increment)
    new_end = datetime.datetime.strftime(new_end_dtg, '%Y-%m-%dT%H:%M:%S.%fZ')

    return new_start, new_end

In [12]:
def next_batch(previous_end, query_terms=query_terms, generate_dates=generate_dates):
    # request 500 tweets at a time
    time_now = datetime.datetime.now()
    print(f"Time of most recent request: {time_now.hour}:{time_now.minute}")

    start, end = generate_dates(previous_end)
    tweet_batch = request_history.main(query_terms, start, end)

    return end, tweet_batch

In [13]:
def create_tweets_df(raw_list):
    list_of_tweets = []
    for tweet in raw_list:
        current_df = pd.DataFrame([tweet])
        list_of_tweets.append(current_df)
    df = pd.concat(list_of_tweets, ignore_index=True)
    return df

In [14]:
all_dfs = []
for _ in range(25):
    try:
        date_, tweets = next_batch(date_)
        df = create_tweets_df(tweets['data'])
        all_dfs.append(df)
    except Exception as e:
        print("Final date: ", date_)
        print("API errors: ", e)
        break

Time of most recent request: 13:19
200
Time of most recent request: 13:19
200
Time of most recent request: 13:19
200
Time of most recent request: 13:20
200
Time of most recent request: 13:20
200
Time of most recent request: 13:20
200
Time of most recent request: 13:20
200
Time of most recent request: 13:20
200
Time of most recent request: 13:20
200
Time of most recent request: 13:20
200
Time of most recent request: 13:20
200
Time of most recent request: 13:20
200
Time of most recent request: 13:20
200
Time of most recent request: 13:20
200
Time of most recent request: 13:20
200
Time of most recent request: 13:20
200
Time of most recent request: 13:20
200
Time of most recent request: 13:20
200
Time of most recent request: 13:20
200
Time of most recent request: 13:20
200
Time of most recent request: 13:20
200
Time of most recent request: 13:20
200
Time of most recent request: 13:20
200
Time of most recent request: 13:20
200
Time of most recent request: 13:20
200


In [15]:
final_df = pd.concat(all_dfs, ignore_index=True)
final_df.shape

(12128, 15)

In [16]:
date_

'2012-12-10T22:00:00.000000Z'

In [17]:
cols = list(final_df.columns)
cols

['possibly_sensitive',
 'referenced_tweets',
 'author_id',
 'text',
 'conversation_id',
 'lang',
 'created_at',
 'public_metrics',
 'entities',
 'edit_history_tweet_ids',
 'source',
 'reply_settings',
 'id',
 'in_reply_to_user_id',
 'attachments']

In [20]:
order = [
    'id',
    'text',
    'author_id',
    'created_at',
    'public_metrics',
    'reply_settings',
    'entities',
    'possibly_sensitive',
    'edit_history_tweet_ids',
    'source',
    'lang',
    'referenced_tweets',
    'conversation_id',
    'in_reply_to_user_id',
    'attachments'    
]

len(order)

15

In [21]:
df = final_df[order]

In [22]:
df.to_csv('./ADD_HUR_DATA/sandy_2012_3day_increment_v2.csv', index=False)
# df.to_csv('./ADD_HUR_DATA/arthur_2014_3day_increment_v2.csv', index=False)
# df.to_csv('./ADD_HUR_DATA/florence_2018_3day_increment.csv', index=False)
# df.to_csv('./ADD_HUR_DATA/barry_2019_3day_increment.csv', index=False)
# df.to_csv('./ADD_HUR_DATA/isaias_2020_3day_increment.csv', index=False)
# df.to_csv('./ADD_HUR_DATA/nicholas_2021_3day_increment.csv', index=False)

In [23]:
df.head()

,id,text,author_id,created_at,public_metrics,reply_settings,entities,possibly_sensitive,edit_history_tweet_ids,source,lang,referenced_tweets,conversation_id,in_reply_to_user_id,attachments
0,247877587320320000,RT @SashaAlexander1: @Sandy_CH hi everyone!!!,71622557,2012-09-18T01:59:56.000Z,"{'retweet_count': 1, 'reply_count': 0, 'like_c...",everyone,"{'mentions': [{'start': 3, 'end': 19, 'usernam...",False,[247877587320320000],Twitter Web Client,en,"[{'type': 'retweeted', 'id': '2475630734838333...",247877587320320000,NaN,NaN
1,247877570606006274,"지드래곤, 3년만에 라디오... '유인나의 볼륨을 높여요' 단독 출연 http:...",501344528,2012-09-18T01:59:52.000Z,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",everyone,"{'urls': [{'start': 41, 'end': 61, 'url': 'htt...",False,[247877570606006274],Twitter for Websites,ko,NaN,247877570606006274,NaN,NaN
2,247877570064949248,RT @Sandy_Darius: @LisaReneeJones --&gt;REVIEW...,464745731,2012-09-18T01:59:52.000Z,"{'retweet_count': 1, 'reply_count': 0, 'like_c...",everyone,"{'urls': [{'start': 107, 'end': 127, 'url': 'h...",False,[247877570064949248],Twitter Web Client,en,"[{'type': 'retweeted', 'id': '2478736788917002...",247877570064949248,NaN,NaN
3,247877565606395904,RT @kevinh801: @OhMyGlob_Sandy cause your a cr...,339666968,2012-09-18T01:59:51.000Z,"{'retweet_count': 1, 'reply_count': 0, 'like_c...",everyone,"{'mentions': [{'start': 3, 'end': 13, 'usernam...",False,[247877565606395904],Twitter for iPhone,en,"[{'type': 'retweeted', 'id': '2478774760370626...",247877565606395904,NaN,NaN
4,247877564532674560,@neewshamm @wacobjilliams not everyone is call...,337478070,2012-09-18T01:59:51.000Z,"{'retweet_count': 0, 'reply_count': 1, 'like_c...",everyone,"{'annotations': [{'start': 93, 'end': 98, 'pro...",False,[247877564532674560],Twitter for iPhone,en,"[{'type': 'replied_to', 'id': '247876134270808...",247876134270808064,284547633,NaN
